# 处理丢失数据

In [2]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

有两种丢失数据：
- None
- np.nan(NaN)

In [1]:
type(None)

NoneType

In [5]:
# NaN 不能参与运算，不是一个数值，只是一个展示符号
# 表达一个空值，要是用np.nan
type(np.nan)

float

In [7]:
# np.nan和任何数运算，得到结果还是nan
np.nan

nan

In [8]:
# 类型强制统一 numpy
np.array([1,2,3, None])

array([1, 2, 3, None], dtype=object)

In [9]:
%timeit np.arange(1e5,dtype=object).sum()

9.11 ms ± 455 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
%timeit np.arange(1e5,dtype=float).sum()

147 µs ± 7.61 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## 1. None

None是Python自带的，其类型为python object。因此，None不能参与到任何计算中。

object类型的运算要比int类型的运算慢得多  
计算不同数据类型求和时间  
%timeit np.arange(1e5,dtype=xxx).sum()

## 2. np.nan（NaN）

np.nan是浮点类型，能参与到计算中。但计算的结果总是NaN。

但可以使用np.nan*()函数来计算nan，此时视nan为0。

## 3. pandas中的None与np.nan

### 1) pandas中None与np.nan都视作np.nan

In [11]:
s = Series(data=np.random.randint(0,10,size=(5)))
s

0    0
1    4
2    5
3    3
4    9
dtype: int32

In [12]:
# pandas对象会把None自动优化成np.nan
s.loc[1] = None

In [13]:
s

0    0.0
1    NaN
2    5.0
3    3.0
4    9.0
dtype: float64

创建DataFrame

In [15]:
df = DataFrame(data=np.random.randint(0,10,size=(3,3)), columns=list("ABC"))
df.loc[1,"B"] = None

In [18]:
df

,A,B,C
0,8,0.0,0
1,6,NaN,8
2,1,2.0,9


In [17]:
# 查看DataFrame的每一列的数据元素类型
df.dtypes

A      int32
B    float64
C      int32
dtype: object

使用DataFrame行索引与列索引修改DataFrame数据

In [ ]:
None --> np.nan

### 2) pandas中None与np.nan的操作

- ``isnull()``
- ``notnull()``
- ``dropna()``: 过滤丢失数据
- ``fillna()``: 填充丢失数据

In [23]:
# 查看哪些列存在空值
df.isnull().any()

A    False
B     True
C    False
dtype: bool

In [26]:
# 查看哪些行存在空值
df.isnull().any(axis=1)

0    False
1     True
2    False
dtype: bool

In [28]:
df.notnull().all()

A     True
B    False
C     True
dtype: bool

(1)判断函数
- ``isnull()``
- ``notnull()``

(2) 过滤函数
- ``dropna()``

In [32]:
# 默认删除的是行
# axis空值删除的方向
# how
df.dropna(axis=1, how='all')

,A,B,C
0,8,0.0,0
1,6,NaN,8
2,1,2.0,9


In [34]:
df.loc[0,"B"] = np.nan
df.loc[2,"B"] = np.nan
df

,A,B,C
0,8,NaN,0
1,6,NaN,8
2,1,NaN,9


In [35]:
df.dropna(axis=1, how='all')

,A,C
0,8,0
1,6,8
2,1,9


可以选择过滤的是行还是列（默认为行）

也可以选择过滤的方式 how = 'all'

(3) 填充函数 Series/DataFrame
- ``fillna()``

In [37]:
# 是用一个数来填充
df.fillna(value=10)

,A,B,C
0,8,10.0,0
1,6,10.0,8
2,1,10.0,9


In [39]:
user_df = DataFrame(data={
    "high":[187,176,180,169],
    "weight":[74,72,80,56],
    "age":[21,25,23,19]
}, index=["lucy","tom","jack","tony"])
user_df.loc["lucy","weight"] = np.nan
user_df.loc["tom","high"] = np.nan
user_df.loc["tony","age"] = np.nan
user_df

,age,high,weight
lucy,21.0,187.0,NaN
tom,25.0,NaN,72.0
jack,23.0,180.0,80.0
tony,NaN,169.0,56.0


In [40]:
user_df.fillna(value=22)

,age,high,weight
lucy,21.0,187.0,22.0
tom,25.0,22.0,72.0
jack,23.0,180.0,80.0
tony,22.0,169.0,56.0


In [42]:
# 通常的办法就是是用每一列的某一个聚合指标来对每一列进行填充
col_mean = user_df.mean()
col_mean

age        23.000000
high      178.666667
weight     69.333333
dtype: float64

In [44]:
user_df

,age,high,weight
lucy,21.0,187.0,NaN
tom,25.0,NaN,72.0
jack,23.0,180.0,80.0
tony,NaN,169.0,56.0


In [43]:
# 可以简写成user_df.fillna(value=user_df.mean())
user_df.fillna(value=col_mean)

,age,high,weight
lucy,21.0,187.000000,69.333333
tom,25.0,178.666667,72.000000
jack,23.0,180.000000,80.000000
tony,23.0,169.000000,56.000000


可以选择前向填充还是后向填充

对于DataFrame来说，还要选择填充的轴axis。记住，对于DataFrame来说：

- axis=0：index/行
- axis=1：columns/列

In [48]:
# 'backfill', 'bfill', 'pad', 'ffill'
user_df.fillna(axis=1, method='ffill')

,age,high,weight
lucy,21.0,187.0,187.0
tom,25.0,25.0,72.0
jack,23.0,180.0,80.0
tony,NaN,169.0,56.0


============================================

练习7：

1. 简述None与NaN的区别

2. 假设张三李四参加模拟考试，但张三因为突然想明白人生放弃了英语考试，因此记为None，请据此创建一个DataFrame,命名为ddd3

3. 老师决定根据用数学的分数填充张三的英语成绩，如何实现？
    用李四的英语成绩填充张三的英语成绩？

============================================

In [49]:
columns = ["语文","数学","英语"]
index = ["张三","李四"]
data = np.random.randint(0,150,size=(2,3))
ddd3 = DataFrame(data=data, index=index, columns=columns)
ddd3

,语文,数学,英语
张三,37,2,76
李四,89,130,73


In [50]:
ddd3.loc["张三","英语"] = np.nan
ddd3

,语文,数学,英语
张三,37,2,NaN
李四,89,130,73.0


In [51]:
ddd3.fillna(method='ffill', axis=1)

,语文,数学,英语
张三,37.0,2.0,2.0
李四,89.0,130.0,73.0


In [52]:
ddd3.fillna(method='bfill', axis=0)

,语文,数学,英语
张三,37,2,73.0
李四,89,130,73.0
